In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk

def local_angle(dx, dy):
    """Calculate the angles between horizontal and vertical operators."""
    return np.mod(np.arctan2(dy, dx), np.pi)

from PIL import Image


import cv2

from genpen.flow.field import *
from genpen.flow.particle import *



%load_ext autoreload
%autoreload 2

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test3.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)vector + self.pg.noise(pt.x, pt.y) * self.noise_mult

# load
img =  rgb2gray(io.imread(Path(image_path)))

In [ ]:

        
img_rescale = rescale(img, image_rescale_factor)
img_renorm = exposure.equalize_adapthist(img_rescale, clip_limit=hist_clip_limit, nbins=hist_nbins)
# img_renorm = img_rescale

In [ ]:
# calc dominant angle
selem = disk(smooth_disk_size)
filt_img = filters.rank.mean(img_renorm, selem)
angle_farid = local_angle(filters.farid_h(filt_img), filters.farid_v(filt_img))

In [ ]:
angle_farid.max()

In [ ]:
remapped_angle_farid = np.interp(angle_farid, xp=[0, np.pi], fp=[np.pi * 0.25, np.pi * 0.75])

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
smoothed_angle = gaussian_filter(remapped_angle_farid, sigma=1)

In [ ]:
plt.imshow(smoothed_angle)

In [ ]:
# make pixel polys
prms = []
for y, row in tqdm(enumerate(img_renorm)):
    for x, intensity in enumerate(row):

        p = gp.centered_box(Point(x, y), width=pixel_width, height=pixel_height)
        a = np.degrees(smoothed_angle[y, x])
        prm = {
            'geometry':p,
            'x':x,
            'y':y,
            'raw_pixel_width':pixel_width,
            'raw_pixel_height':pixel_height,
            'intensity': intensity,
            'angle':a,
            'group': 'raw_hatch_pixel',

        }
        prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 15 * 15

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
qpg = QuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5)
qpg.make_grid()

In [ ]:
# # evenly spaced grid
# bins, grid = gp.overlay_grid(new_drawbox, xstep=2.5, ystep=2.5, flatmesh=True)
# xs, ys = grid
# pts = [Point(x,y) for x,y in zip(xs, ys)]

# # random
# pts = gp.get_random_points_in_polygon(new_drawbox, 4000)

n_points = 5000
pts = []
pix_p =  np.interp(scaled_hatch_pixels['intensity'], [0, 1], [0.9, 0.1])
pix_p /= pix_p.sum()
for ii in range(n_points):
    pix = np.random.choice(scaled_hatch_pixels.index, p=pix_p)
    pt = gp.get_random_point_in_polygon(scaled_hatch_pixels.loc[pix, 'geometry'])
    pts.append(pt)

# # circle    
# rad = 50
# n_points = 100
# circ = new_drawbox.centroid.buffer(rad).boundary
# pts = [circ.interpolate(d, normalized=True) for d in np.linspace(0, 1, n_points)]

In [ ]:
vps = []
for p in pts:
    vp = VectorParticle(pos=p, vector=np.array([0, 1]), grid = qpg, stepsize=1)
    vp.momentum_factor = np.interp(vp.y, [db.bottom, db.top], [2, 2])
    vps.append(vp)

In [ ]:
for vp in tqdm(vps):
    for ii in range(29):
        vp.momentum_factor = np.interp(vp.y, [db.bottom, db.top], [2, 2])
        vp.step()

In [ ]:
vps = [vp for vp in vps if len(vp.pts) > 1]

lss = [LineString(vp.pts) for vp in vps]

lss = gp.merge_LineStrings(lss)

# blss = lss.buffer(0.01, join_style=2, cap_style=2).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(lss)
sk.display()

In [ ]:
merge_tolerances = [0.2, 0.3, 0.4, 0.5, 0.6]
simplify_tolerances = [0.2]

In [ ]:
# sk.vpype('splitall')
        
    
for tolerance in tqdm(merge_tolerances):
    sk.vpype(f'linemerge --tolerance {tolerance}mm')

for tolerance in tqdm(simplify_tolerances):
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')

sk.vpype('linesort')
sk.display()

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# w subdivide

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test12.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

# load
img =  rgb2gray(io.imread(Path(image_path)))

In [ ]:
xgen = ss.uniform(loc=0.45, scale=0.0).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.8, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)
# split_func = functools.partial(gp.random_bezier_subdivide, x0=0.19, x1=0.85, n_eval_points=50)
# splits = gp.recursive_split_frac_buffer(
#     drawbox, 
#     split_func=split_func,
#     p_continue=0.7, 
#     depth=0, 
#     depth_limit=8,
#     buffer_frac=-0.0
# )


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

    a = np.random.uniform(0, 240)
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'intensity': 1,

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.0001, noise_mult=1, verbose=True)

qpg.make_grid()

# qpg = QuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5)
# qpg.make_grid()

In [ ]:
# # evenly spaced grid
# bins, grid = gp.overlay_grid(new_drawbox, xstep=2.5, ystep=2.5, flatmesh=True)
# xs, ys = grid
# pts = [Point(x,y) for x,y in zip(xs, ys)]

# # random
# pts = gp.get_random_points_in_polygon(new_drawbox, 4000)

# n_points = 5000
# pts = []
# pix_p =  np.interp(scaled_hatch_pixels['intensity'], [0, 1], [0.9, 0.1])
# pix_p /= pix_p.sum()
# for ii in range(n_points):
#     pix = np.random.choice(scaled_hatch_pixels.index, p=pix_p)
#     pt = gp.get_random_point_in_polygon(scaled_hatch_pixels.loc[pix, 'geometry'])
#     pts.append(pt)

# # circle    
# rad = 50
# n_points = 100
# circ = new_drawbox.centroid.buffer(rad).boundary
# pts = [circ.interpolate(d, normalized=True) for d in np.linspace(0, 1, n_points)]

In [ ]:
def get_random_line_in_polygon(polygon, max_dist=None, min_dist=None):
    pt0 = gp.get_random_point_in_polygon(polygon)
    pt1 = gp.get_random_point_in_polygon(polygon)
    
    if max_dist is not None:
        while pt0.distance(pt1) > max_dist:
            pt1 = gp.get_random_point_in_polygon(polygon)
            
    if min_dist is not None:
        while pt0.distance(pt1) < min_dist:
            pt1 = gp.get_random_point_in_polygon(polygon)
    
    return LineString([pt0, pt1])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.8, verbose=False)
qpg.make_grid()

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_lines = 900
for ii in tqdm(range(n_lines)):
    ls = get_random_line_in_polygon(poly, min_dist = 10, max_dist=400)
    new_pts = [ls.interpolate(d) for d in np.linspace(0, ls.length, np.random.randint(1,32))]
    vps = [VectorParticle(pos=pt, vector=np.random.uniform(-1,1,size=2), grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0)) for pt in new_pts]
    for vp in vps:
        for ii in range(10):
            vp.step()
    vps = [vp for vp in vps if len(vp.pts) > 1]
    ls = gp.merge_LineStrings([LineString(vp.pts) for vp in vps])
    lss.append(ls)

In [ ]:
blss = gp.merge_LineStrings(lss).buffer(0.1, cap_style=2, join_style=2)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_lines = 900
for ii in tqdm(range(n_lines)):
    ls = get_random_line_in_polygon(poly, min_dist = 10, max_dist=400)
    new_pts = [ls.interpolate(d) for d in np.linspace(0, ls.length, np.random.randint(1,32))]
    vps = [VectorParticle(pos=pt, vector=np.random.uniform(-1,1,size=2), grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0)) for pt in new_pts]
    for vp in vps:
        for ii in range(10):
            vp.step()
    vps = [vp for vp in vps if len(vp.pts) > 1]
    ls = gp.merge_LineStrings([LineString(vp.pts) for vp in vps])
    lss.append(ls)

In [ ]:
blss2 = gp.merge_LineStrings(lss).buffer(0.1, cap_style=2, join_style=2)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
# for ii, ls in enumerate(lss):
#     sk.stroke(ii + 1)
#     sk.geometry(ls)
sk.stroke(1)
sk.geometry(blss)
sk.stroke(2)
sk.geometry(blss2)
    
sk.display()

In [ ]:
merge_tolerances = [0.2, 0.3, 0.4, 0.5, 1]
simplify_tolerances = [0.2]

In [ ]:
sk.vpype('splitall')
        
    
for tolerance in tqdm(merge_tolerances):
    sk.vpype(f'linemerge --tolerance {tolerance}mm')

for tolerance in tqdm(simplify_tolerances):
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')

sk.vpype('linesort')
sk.display()

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# w subdivide

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test14.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

# load
img =  rgb2gray(io.imread(Path(image_path)))

In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.05).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=1, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)
# split_func = functools.partial(gp.random_bezier_subdivide, x0=0.19, x1=0.85, n_eval_points=50)
# splits = gp.recursive_split_frac_buffer(
#     drawbox, 
#     split_func=split_func,
#     p_continue=0.7, 
#     depth=0, 
#     depth_limit=8,
#     buffer_frac=-0.0
# )


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
all_bps = gp.Shape(bps)

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
def get_random_line_in_polygon(polygon, max_dist=None, min_dist=None):
    pt0 = gp.get_random_point_in_polygon(polygon)
    pt1 = gp.get_random_point_in_polygon(polygon)
    
    if max_dist is not None:
        while pt0.distance(pt1) > max_dist:
            pt1 = gp.get_random_point_in_polygon(polygon)
            
    if min_dist is not None:
        while pt0.distance(pt1) < min_dist:
            pt1 = gp.get_random_point_in_polygon(polygon)
    
    return LineString([pt0, pt1])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.5, verbose=False)
qpg.make_grid()

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_lines = 6000
for ii in tqdm(range(n_lines)):
    ls = get_random_line_in_polygon(poly, min_dist = 10, max_dist=400)
    new_pts = [ls.interpolate(d) for d in np.linspace(0, ls.length, np.random.randint(1,2))]
    vps = [VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0)) for pt in new_pts]
    for vp in vps:
        for ii in range(15):
            vp.step()
    vps = [vp for vp in vps if len(vp.pts) > 1]
    ls = gp.merge_LineStrings([LineString(vp.pts) for vp in vps])
    lss.append(ls)

In [ ]:
blss = gp.merge_LineStrings(lss).buffer(0.2, cap_style=2, join_style=2)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
# for ii, ls in enumerate(lss):
#     sk.stroke(ii + 1)
#     sk.geometry(ls)
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

In [ ]:
merge_tolerances = [0.2, 0.3, 0.4, 0.5, 1]
simplify_tolerances = [0.2]

In [ ]:
sk.vpype('splitall')
        
    
for tolerance in tqdm(merge_tolerances):
    sk.vpype(f'linemerge --tolerance {tolerance}mm')

for tolerance in tqdm(simplify_tolerances):
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')

sk.vpype('linesort')
sk.display()

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# spiral start

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test15.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

# load
img =  rgb2gray(io.imread(Path(image_path)))

In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.05).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=1, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)
# split_func = functools.partial(gp.random_bezier_subdivide, x0=0.19, x1=0.85, n_eval_points=50)
# splits = gp.recursive_split_frac_buffer(
#     drawbox, 
#     split_func=split_func,
#     p_continue=0.7, 
#     depth=0, 
#     depth_limit=8,
#     buffer_frac=-0.0
# )


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
all_bps = gp.Shape(bps)

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.5, verbose=False)
qpg.make_grid()

In [ ]:
spiral_angle_max = np.pi * 200
spiral_angle_min = 0
spiral_angle_spacing = np.pi * 0.053
sp_angle_range = np.arange(spiral_angle_min, spiral_angle_max, spiral_angle_spacing)
spiral_distances = np.linspace(0, 100, len(sp_angle_range))

start_points = [Point(np.cos(a) * d, np.sin(a) * d) for a, d in zip(sp_angle_range, spiral_distances)]

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_steps = 8
for pt in tqdm(start_points):
    
    vp = VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
    lss.append(ls)

In [ ]:
for ls in lss:
    ls

In [ ]:
blss = gp.merge_LineStrings(lss).buffer(0.25, cap_style=2, join_style=2)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
# for ii, ls in enumerate(lss):
#     sk.stroke(ii + 1)
#     sk.geometry(ls)
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

In [ ]:
merge_tolerances = [0.2, 0.3, 0.4, 0.5, 1]
simplify_tolerances = [0.2]

In [ ]:
sk.vpype('splitall')
        
    
for tolerance in tqdm(merge_tolerances):
    sk.vpype(f'linemerge --tolerance {tolerance}mm')

for tolerance in tqdm(simplify_tolerances):
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')

sk.vpype('linesort')
sk.display()

In [ ]:
filename = 'vp_test17.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# spiral start buffer shaded

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test18.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

# load
img =  rgb2gray(io.imread(Path(image_path)))

In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.05).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=1, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)
# split_func = functools.partial(gp.random_bezier_subdivide, x0=0.19, x1=0.85, n_eval_points=50)
# splits = gp.recursive_split_frac_buffer(
#     drawbox, 
#     split_func=split_func,
#     p_continue=0.7, 
#     depth=0, 
#     depth_limit=8,
#     buffer_frac=-0.0
# )


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
all_bps = gp.Shape(bps)

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.5, verbose=False)
qpg.make_grid()

In [ ]:
spiral_angle_max = np.pi * 200
spiral_angle_min = 0
spiral_angle_spacing = np.pi * 0.063
sp_angle_range = np.arange(spiral_angle_min, spiral_angle_max, spiral_angle_spacing)
spiral_distances = np.linspace(0, 100, len(sp_angle_range))

start_points = [Point(np.cos(a) * d, np.sin(a) * d) for a, d in zip(sp_angle_range, spiral_distances)]

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_steps = 5
for pt in tqdm(start_points):
    
    vp = VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
    lss.append(ls)

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=1.1).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.35, high=-0.25)
d_translate_factor_gen = ss.uniform(loc=0.6, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ii, l in enumerate(tqdm(lss[:])):
    p = l.buffer(0.1, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    angles_gen = gp.make_callable(sp_angle_range[ii]-90)
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
blss = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
# for ii, ls in enumerate(lss):
#     sk.stroke(ii + 1)
#     sk.geometry(ls)
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

In [ ]:
merge_tolerances = [0.2, 0.3, 0.4, 0.5, 1]
simplify_tolerances = [0.2]

In [ ]:
sk.vpype('splitall')
        
    
for tolerance in tqdm(merge_tolerances):
    sk.vpype(f'linemerge --tolerance {tolerance}mm')

for tolerance in tqdm(simplify_tolerances):
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')

sk.vpype('linesort')
sk.display()

In [ ]:
filename = 'vp_test28.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)